In [1]:
import pandas as pd
import numpy as np
import json
from bs4 import BeautifulSoup
import requests
import re
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

In [2]:
from dk_weekly_update import DkWeeklyUpdate as dk

In [2]:
df = pd.read_csv('input/te/te_loc_cleaned.csv')
df['year'].unique()

array([2014, 2016, 2017, 2018, 2012, 2013, 2015], dtype=int64)

In [9]:
wk8 = pd.read_csv('input/te/te_wk8.csv')
wk8

,Rank,Name,Team,Position,Week,Opponent,ReceivingTargets,Receptions,ReceptionPercentage,ReceivingYards,...,ReceivingYardsPerTarget,ReceivingYardsPerReception,RushingAttempts,RushingYards,RushingYardsPerAttempt,RushingTouchdowns,Fumbles,FumblesLost,FantasyPointsPerGameDraftKings,FantasyPointsDraftKings
0,1,Travis Kelce,KC,TE,8,DEN,10,6,60.0,79,...,7.9,13.2,0,0,0,0,0,0,19.9,19.9
1,2,Jack Doyle,IND,TE,8,OAK,7,6,85.7,70,...,10.0,11.7,0,0,0,0,0,0,19.0,19.0
2,3,Jordan Thomas,HOU,TE,8,MIA,4,4,100.0,29,...,7.2,7.2,0,0,0,0,0,0,18.9,18.9
3,4,Jared Cook,OAK,TE,8,IND,5,4,80.0,74,...,14.8,18.5,0,0,0,0,0,0,17.4,17.4
4,5,OJ Howard,TB,TE,8,CIN,4,4,100.0,68,...,17.0,17.0,0,0,0,0,0,0,16.8,16.8
5,6,Greg Olsen,CAR,TE,8,BAL,4,4,100.0,56,...,14.0,14.0,0,0,0,0,0,0,15.6,15.6
6,7,Evan Engram,NYG,TE,8,WAS,9,5,55.6,25,...,2.8,5.0,0,0,0,0,0,0,13.5,13.5
7,8,Ed Dickson,SEA,TE,8,DET,2,2,100.0,54,...,27.0,27.0,0,0,0,0,0,0,13.4,13.4
8,9,Eric Ebron,IND,TE,8,OAK,3,3,100.0,37,...,12.3,12.3,0,0,0,0,0,0,12.7,12.7
9,10,Zach Ertz,PHI,TE,8,JAX,6,4,66.7,26,...,4.3,6.5,0,0,0,0,0,0,12.6,12.6


In [3]:
df.tail()

,games,year,Name,Team,Week,Opponent,tar,rec,yds,td,yds/tar,yds/rec,def_rk,h/a_1,surface_0,points
9151,36,2018,Tyler Higbee,LAR,7,SF,1,0,0,0,0.0,0.0,23,0.0,1.0,0.0
9152,3,2018,Johnny Mundt,LAR,7,SF,0,0,0,0,0.0,0.0,23,0.0,1.0,0.0
9153,1,2018,Ross Dwelley,SF,7,LAR,0,0,0,0,0.0,0.0,24,1.0,1.0,0.0
9154,82,2018,Logan Paulsen,ATL,7,NYG,0,0,0,0,0.0,0.0,9,1.0,0.0,0.0
9155,20,2018,Scott Simonson,NYG,7,ATL,1,0,0,0,0.0,0.0,19,0.0,0.0,0.0


In [4]:
def_2018 = pd.read_csv('input/def/fantasy-football-points-allowed-defense-by-position.csv')
def_2018.head()


,Rank,Name,Team,Position,Week,Opponent,DraftKingsQuarterbackFantasyPointsAllowedAverage,DraftKingsRunningbackFantasyPointsAllowedAverage,DraftKingsWideReceiverFantasyPointsAllowedAverage,DraftKingsTightEndFantasyPointsAllowedAverage,DraftKingsKickerFantasyPointsAllowedAverage,DraftKingsFantasyPointsAllowedAverage
0,1,Chicago Bears,CHI,DST,8,NYJ,12.32,10.0,20.2,8.6,5,56.12
1,2,Arizona Cardinals,ARI,DST,8,SF,12.50,9.8,28.3,10.7,7,68.30
2,3,New England Patriots,NE,DST,8,BUF,10.42,22.0,29.0,9.0,9,79.42
3,4,Pittsburgh Steelers,PIT,DST,8,CLE,15.10,16.5,29.6,12.4,7,80.60
4,5,Minnesota Vikings,MIN,DST,8,NO,10.86,38.8,17.9,0.0,16,83.56


In [5]:
location = pd.read_csv('input/location.csv')
location.tail()

,Unnamed: 0,date,day,week,home,away,year
1653,116,"October 28, 2018",Sun,8,OAK,IND,2018
1654,117,"October 28, 2018",Sun,8,LAR,GB,2018
1655,118,"October 28, 2018",Sun,8,ARI,SF,2018
1656,119,"October 28, 2018",Sun,8,MIN,NO,2018
1657,120,"October 29, 2018",Mon,8,BUF,NE,2018


In [10]:
wk8.rename(columns ={  'Receptions':'rec',
                     'ReceivingTargets':'tar',
                      'ReceivingYards':'yds',
                     'ReceivingYardsPerReception':'yds/rec',
                     'ReceivingYardsPerTarget':'yds/tar',
                      'ReceivingTouchdowns':'td',                    
                      'FantasyPointsDraftKings':'points'},inplace=True)
wk8.columns


Index(['Rank', 'Name', 'Team', 'Position', 'Week', 'Opponent', 'tar', 'rec',
       'ReceptionPercentage', 'yds', 'td', 'ReceivingLong', 'yds/tar',
       'yds/rec', 'RushingAttempts', 'RushingYards', 'RushingYardsPerAttempt',
       'RushingTouchdowns', 'Fumbles', 'FumblesLost',
       'FantasyPointsPerGameDraftKings', 'points'],
      dtype='object')

In [6]:
position_dict = {'qb':'DraftKingsQuarterbackFantasyPointsAllowedAverage',
                    'wr':'DraftKingsWideReceiverFantasyPointsAllowedAverage',
                    'rb':'DraftKingsRunningbackFantasyPointsAllowedAverage',
                    'te':'DraftKingsTightEndFantasyPointsAllowedAverage'}

team_dict = {'New England Patriots':'NE',
            'Green Bay Packers':'GB',
            'Tampa Bay Buccaneers':'TB',
            'Kansas City Chiefs':'KC',
            'San Diego Chargers':'SD',
            'St. Louis Rams':'STL',
            'New Orleans Saints':'NO',
            'New York Jets':'NYJ',
            'New York Giants':'NYG',
            'San Francisco 49ers':'SF',
            'Los Angeles Rams':'LAR',
            'Jacksonville Jaguars':'JAX',
            'Los Angeles Chargers':'LAC'}

In [7]:
df.head()

,games,year,Name,Team,Week,Opponent,tar,rec,yds,td,yds/tar,yds/rec,def_rk,h/a_1,surface_0,points
0,1,2014,AD Denham,HOU,14,JAX,0,0,0,0,0.0,0.0,30,0.0,1.0,0.0
1,2,2014,AD Denham,HOU,15,IND,0,0,0,0,0.0,0.0,5,0.0,0.0,0.0
2,3,2014,AD Denham,HOU,16,BAL,0,0,0,0,0.0,0.0,4,1.0,0.0,0.0
3,4,2014,AD Denham,HOU,17,JAX,0,0,0,0,0.0,0.0,30,1.0,0.0,0.0
4,1,2016,AJ Derby,NE,1,ARI,0,0,0,0,0.0,0.0,1,0.0,1.0,0.0


In [8]:
df.columns

Index(['games', 'year', 'Name', 'Team', 'Week', 'Opponent', 'tar', 'rec',
       'yds', 'td', 'yds/tar', 'yds/rec', 'def_rk', 'h/a_1', 'surface_0',
       'points'],
      dtype='object')

In [11]:
ru_cols = [col for col in wk8.columns if 'Rushing'in col]
wk8.drop(ru_cols, axis=1, inplace=True)

In [12]:
wk8.drop(['FantasyPointsPerGameDraftKings', 'Rank','Position', 'Fumbles','FumblesLost','ReceptionPercentage','ReceptionPercentage','ReceivingLong'],axis=1, inplace=True)
wk8.columns

Index(['Name', 'Team', 'Week', 'Opponent', 'tar', 'rec', 'yds', 'td',
       'yds/tar', 'yds/rec', 'points'],
      dtype='object')

In [13]:
wk8['year']=2018

In [14]:
def_2018.head()

,Rank,Name,Team,Position,Week,Opponent,DraftKingsQuarterbackFantasyPointsAllowedAverage,DraftKingsRunningbackFantasyPointsAllowedAverage,DraftKingsWideReceiverFantasyPointsAllowedAverage,DraftKingsTightEndFantasyPointsAllowedAverage,DraftKingsKickerFantasyPointsAllowedAverage,DraftKingsFantasyPointsAllowedAverage
0,1,Chicago Bears,CHI,DST,8,NYJ,12.32,10.0,20.2,8.6,5,56.12
1,2,Arizona Cardinals,ARI,DST,8,SF,12.50,9.8,28.3,10.7,7,68.30
2,3,New England Patriots,NE,DST,8,BUF,10.42,22.0,29.0,9.0,9,79.42
3,4,Pittsburgh Steelers,PIT,DST,8,CLE,15.10,16.5,29.6,12.4,7,80.60
4,5,Minnesota Vikings,MIN,DST,8,NO,10.86,38.8,17.9,0.0,16,83.56


In [15]:
wk8.head()

,Name,Team,Week,Opponent,tar,rec,yds,td,yds/tar,yds/rec,points,year
0,Travis Kelce,KC,8,DEN,10,6,79,1,7.9,13.2,19.9,2018
1,Jack Doyle,IND,8,OAK,7,6,70,1,10.0,11.7,19.0,2018
2,Jordan Thomas,HOU,8,MIA,4,4,29,2,7.2,7.2,18.9,2018
3,Jared Cook,OAK,8,IND,5,4,74,1,14.8,18.5,17.4,2018
4,OJ Howard,TB,8,CIN,4,4,68,1,17.0,17.0,16.8,2018


In [17]:
df.shape

(10196, 16)

## Add Defensive Ranking

In [20]:
wk8= add_defensive_rank(wk8, def_2018, 'te')
wk8

,Name,Team,Week,Opponent,tar,rec,yds,td,yds/tar,yds/rec,points,year,def_rk
0,Travis Kelce,KC,8,DEN,10,6,79,1,7.9,13.2,19.9,2018,26
1,Jack Doyle,IND,8,OAK,7,6,70,1,10.0,11.7,19.0,2018,28
2,Jordan Thomas,HOU,8,MIA,4,4,29,2,7.2,7.2,18.9,2018,23
3,Jared Cook,OAK,8,IND,5,4,74,1,14.8,18.5,17.4,2018,21
4,OJ Howard,TB,8,CIN,4,4,68,1,17.0,17.0,16.8,2018,24
5,Greg Olsen,CAR,8,BAL,4,4,56,1,14.0,14.0,15.6,2018,22
6,Evan Engram,NYG,8,WAS,9,5,25,1,2.8,5.0,13.5,2018,17
7,Ed Dickson,SEA,8,DET,2,2,54,1,27.0,27.0,13.4,2018,20
8,Eric Ebron,IND,8,OAK,3,3,37,1,12.3,12.3,12.7,2018,28
9,Zach Ertz,PHI,8,JAX,6,4,26,1,4.3,6.5,12.6,2018,27


In [22]:
loc= location[location['year']==2018]

loc = loc[loc['week']==8]
loc.shape

(14, 7)

## Home or Away

In [23]:
loc_wk = {}
home_lst = []
for week in loc['week'].unique():
    loc_wk[week]  = loc[loc['week']==week]
for idx, team in enumerate(wk8['Team']):
    if team in list(loc['home']):
        home_lst.append('home')           
    else:
        home_lst.append('away')
wk8['h/a'] = home_lst

In [25]:
wk8.head()

,Name,Team,Week,Opponent,tar,rec,yds,td,yds/tar,yds/rec,points,year,def_rk,h/a
0,Travis Kelce,KC,8,DEN,10,6,79,1,7.9,13.2,19.9,2018,26,home
1,Jack Doyle,IND,8,OAK,7,6,70,1,10.0,11.7,19.0,2018,28,away
2,Jordan Thomas,HOU,8,MIA,4,4,29,2,7.2,7.2,18.9,2018,23,home
3,Jared Cook,OAK,8,IND,5,4,74,1,14.8,18.5,17.4,2018,21,home
4,OJ Howard,TB,8,CIN,4,4,68,1,17.0,17.0,16.8,2018,24,away


## Surface

In [26]:
surface_dict={'KC':'grass',
             'CAR':'grass',
              'WAS':'grass',
              'CLE':'grass',
              'MIA':'grass',
              'PIT':'grass',
              'GB':'grass',
              'SF':'grass',
              'PHI':'grass',
              'LAR':'grass',
              'BAL': 'grass',
              'TEN':'grass',
              'OAK':'grass',
              'LAC':'grass',
              'TB':'grass',
              'CHI':'grass',
              'DEN':'grass',
              'ARI':'grass',
              'DAL':'turf',
              'SEA':'turf',
              'DET':'turf',
              'ATL':'turf',
              'NE':'turf',
              'IND':'turf',
              'NO':'turf',
              'NYG':'turf',
              'NYJ':'turf',
              'HOU':'turf',
              'BUF':'turf',
              'CIN':'turf',
              'MIN':'turf',
              'JAX':'grass'
              
             }

In [27]:


# Add the type of field the player palyed on by mapping the field on which the game was played to the surface dictionary

surface = []
for idx, team in enumerate(wk8['Team']):
    if wk8.iloc[idx]['h/a']=='home':
        surface.append(surface_dict[team])
    else:
        surface.append(surface_dict[wk8.iloc[idx]['Opponent']])


wk8['surface'] = surface

In [28]:
wk8.head()

,Name,Team,Week,Opponent,tar,rec,yds,td,yds/tar,yds/rec,points,year,def_rk,h/a,surface
0,Travis Kelce,KC,8,DEN,10,6,79,1,7.9,13.2,19.9,2018,26,home,grass
1,Jack Doyle,IND,8,OAK,7,6,70,1,10.0,11.7,19.0,2018,28,away,grass
2,Jordan Thomas,HOU,8,MIA,4,4,29,2,7.2,7.2,18.9,2018,23,home,turf
3,Jared Cook,OAK,8,IND,5,4,74,1,14.8,18.5,17.4,2018,21,home,grass
4,OJ Howard,TB,8,CIN,4,4,68,1,17.0,17.0,16.8,2018,24,away,turf


## Encode

In [29]:
wk8 = one_hot(wk8)

In [30]:
wk8.drop(['h/a_0','surface_1','surface','h/a','surface_encoded','h/a_encoded'], axis=1, inplace=True)
wk8.shape

(82, 15)

In [31]:
wk8.columns

Index(['Name', 'Team', 'Week', 'Opponent', 'tar', 'rec', 'yds', 'td',
       'yds/tar', 'yds/rec', 'points', 'year', 'def_rk', 'h/a_1', 'surface_0'],
      dtype='object')

In [32]:
wk8['year']=2018
wk8.shape

(82, 15)

In [33]:
df.columns

Index(['games', 'year', 'Name', 'Team', 'Week', 'Opponent', 'tar', 'rec',
       'yds', 'td', 'yds/tar', 'yds/rec', 'def_rk', 'h/a_1', 'surface_0',
       'points'],
      dtype='object')

In [35]:
cols = wk8.columns.tolist()
cols = cols[-4:-3] + cols[:10] + cols[-3:] + cols[-5:-4]


['Name',
 'Team',
 'Week',
 'Opponent',
 'tar',
 'rec',
 'yds',
 'td',
 'yds/tar',
 'yds/rec',
 'points',
 'year',
 'def_rk',
 'h/a_1',
 'surface_0']

In [133]:
wk7.shape

(82, 16)

In [134]:
df.shape

(9074, 16)

In [115]:
df = pd.concat([df, wk1_6])
df.year.unique()

array([2014, 2016, 2017, 2012, 2013, 2015, 2018], dtype=int64)

## Games

In [119]:
df.head()

,year,Name,Team,Week,Opponent,tar,rec,yds,td,yds/tar,yds/rec,def_rk,h/a_1,surface_0,points
0,2014,AD Denham,HOU,14,JAX,0,0,0,0,0.0,0.0,30,0.0,1.0,0.0
1,2014,AD Denham,HOU,15,IND,0,0,0,0,0.0,0.0,5,0.0,0.0,0.0
2,2014,AD Denham,HOU,16,BAL,0,0,0,0,0.0,0.0,4,1.0,0.0,0.0
3,2014,AD Denham,HOU,17,JAX,0,0,0,0,0.0,0.0,30,1.0,0.0,0.0
4,2016,AJ Derby,NE,1,ARI,0,0,0,0,0.0,0.0,1,0.0,1.0,0.0


In [36]:
games_lst = []
temp = wk8.set_index('Name')
for name in temp.index:
    if name in list(df['Name'].unique()):
        games_lst.append(df[df['Name']==name]['games'].max()+1)
    else:
        games_lst.append(1)
    

In [37]:
wk8['games'] = games_lst

In [38]:
wk8.head()

,Name,Team,Week,Opponent,tar,rec,yds,td,yds/tar,yds/rec,points,year,def_rk,h/a_1,surface_0,games
0,Travis Kelce,KC,8,DEN,10,6,79,1,7.9,13.2,19.9,2018,26,1.0,1.0,71
1,Jack Doyle,IND,8,OAK,7,6,70,1,10.0,11.7,19.0,2018,28,0.0,1.0,81
2,Jordan Thomas,HOU,8,MIA,4,4,29,2,7.2,7.2,18.9,2018,23,1.0,0.0,5
3,Jared Cook,OAK,8,IND,5,4,74,1,14.8,18.5,17.4,2018,21,1.0,1.0,94
4,OJ Howard,TB,8,CIN,4,4,68,1,17.0,17.0,16.8,2018,24,0.0,0.0,20


## Reshape

In [39]:
wk8['year']=2018

In [40]:
wk8.columns

Index(['Name', 'Team', 'Week', 'Opponent', 'tar', 'rec', 'yds', 'td',
       'yds/tar', 'yds/rec', 'points', 'year', 'def_rk', 'h/a_1', 'surface_0',
       'games'],
      dtype='object')

In [41]:
cols = df.columns.tolist()
cols


['games',
 'year',
 'Name',
 'Team',
 'Week',
 'Opponent',
 'tar',
 'rec',
 'yds',
 'td',
 'yds/tar',
 'yds/rec',
 'def_rk',
 'h/a_1',
 'surface_0',
 'points']

In [46]:
cols = wk8.columns.tolist()
cols = cols[-1:] + cols[-5:-4] + cols[:10] + cols[-4:-1] + cols[-6:-5]
wk8 = wk8[cols]
wk8.head()

,games,year,Name,Team,Week,Opponent,tar,rec,yds,td,yds/tar,yds/rec,def_rk,h/a_1,surface_0,points
0,71,2018,Travis Kelce,KC,8,DEN,10,6,79,1,7.9,13.2,26,1.0,1.0,19.9
1,81,2018,Jack Doyle,IND,8,OAK,7,6,70,1,10.0,11.7,28,0.0,1.0,19.0
2,5,2018,Jordan Thomas,HOU,8,MIA,4,4,29,2,7.2,7.2,23,1.0,0.0,18.9
3,94,2018,Jared Cook,OAK,8,IND,5,4,74,1,14.8,18.5,21,1.0,1.0,17.4
4,20,2018,OJ Howard,TB,8,CIN,4,4,68,1,17.0,17.0,24,0.0,0.0,16.8


In [47]:
df.shape

(9156, 16)

In [49]:
wk8.shape

(82, 16)

## Concatenate

In [50]:
te = pd.concat([df,wk8])
te.shape

(9238, 16)

In [51]:
te.tail()

,games,year,Name,Team,Week,Opponent,tar,rec,yds,td,yds/tar,yds/rec,def_rk,h/a_1,surface_0,points
77,77,2018,Josh Hill,NO,8,MIN,0,0,0,0,0.0,0.0,1,0.0,0.0,0.0
78,31,2018,David Morgan,MIN,8,NO,1,0,0,0,0.0,0.0,10,1.0,0.0,0.0
79,3,2018,Tyler Conklin,MIN,8,NO,0,0,0,0,0.0,0.0,10,1.0,0.0,0.0
80,75,2018,Dwayne Allen,NE,8,BUF,0,0,0,0,0.0,0.0,7,0.0,0.0,0.0
81,18,2018,Logan Thomas,BUF,8,NE,0,0,0,0,0.0,0.0,13,1.0,0.0,0.0


In [52]:
te.to_csv('input/te/te_loc_cleaned.csv', index=False)

In [16]:
# Parse upcoming week file by position

def clean_current(csv):# Read in files

    df_upcoming = pd.read_csv(csv)
    def_2018 = pd.read_csv('input/def/fantasy-football-points-allowed-defense-by-position.csv')

    # Obtain home and away attributes
    df_upcoming['year']=2018
    df_upcoming['home'] = df_upcoming['Game Info'].apply(lambda x: (x.split('@')[1]).split()[0])
    df_upcoming['away'] = df_upcoming['Game Info'].apply(lambda x: (x.split('@')[0]))

    rb = []
    wr = []
    te = []
    qb= []
    for idx,pos in enumerate(df_upcoming['Position']):
        if pos == 'RB':
            rb.append(idx)
        elif pos =='WR':
            wr.append(idx)
        elif pos =='TE':
            te.append(idx)
        elif pos =='QB':
            qb.append(idx)


    # Sort current week dataframe

    qb_df = df_upcoming.iloc[qb]
    rb_df = df_upcoming.iloc[rb]
    wr_df = df_upcoming.iloc[wr]
    te_df = df_upcoming.iloc[te]
    
    return qb_df,wr_df,te_df, rb_df

In [17]:
# Prepare defense dataframe by sorting by relevant attribute reseting the indexand using rank as the reset dataframe index +1

def add_defensive_rank(df, df1, position):
    df1.sort_values(position_dict[position], inplace=True)
    df1.reset_index(inplace=True, drop=True)
    df1['Rank']= df1.index+1
    df1.index=df1['Team']

# Map defensive rnking to eack player in the statist  ical dataframe 

    df['def_rk'] = list(map(lambda x: df1.loc[x]['Rank'], df['Opponent']))
    
    return df


In [18]:
# A function designed to add the Draftkings fantasy football salary to every players statistics dataframe

def add_salary(df,df2):
    df = df.sort_values(['Week','PlayerID'])
    df = df.drop(['FantasyPointsPerGameDraftKings','Rank','Position'], axis=1)
    df.reset_index(inplace=True, drop=True)
    df2 = df2.sort_values(['Week','PlayerID'])
    df2 = df2.drop(['Rank','Position'], axis=1)
    df2.reset_index(inplace=True, drop=True)
    df['salary'] = df2['DraftKingsSalary']
    
    return df

    

In [19]:
# One hot encode all non numeric attributes


def one_hot(df):
    df.reset_index(inplace=True, drop=True)
    encoder = LabelEncoder()
    df['h/a_encoded'] = encoder.fit_transform(df['h/a'])
    df['surface_encoded'] = encoder.fit_transform(df['surface'])


    home =OneHotEncoder()
    surface = OneHotEncoder()

    X = home.fit_transform(df['h/a_encoded'].values.reshape(-1,1)).toarray()
    Xs = surface.fit_transform(df['surface_encoded'].values.reshape(-1,1)).toarray()

    dfOneHot = pd.DataFrame(X, columns = ["h/a_"+str(int(i)) for i in range(X.shape[1])])
    df = pd.concat([df, dfOneHot], axis=1)


    dfOneHot = pd.DataFrame(Xs, columns = ['surface_'+str(int(i)) for i in range(Xs.shape[1])])
    df = pd.concat([df, dfOneHot], axis=1)

    return df

In [97]:
csv = 'input/qb/DKSalaries.csv'
qb7, wr7, te7, rb7 = clean_current(csv)
